In [5]:
import sys
import os

# Add the 'src' directory to Python's module search path
src_dir = os.path.abspath(os.path.join("..", "src"))  # Goes up to 'Challenge #1', then into 'src'
sys.path.append(src_dir)

In [15]:
import pandas as pd
from torchvision import transforms
from pathlib import Path
from essentials import TinyVGG
import torch
from tqdm import tqdm
from PIL import Image


In [3]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

idx_to_class = {0: 'Alluvial soil', 1: 'Black Soil', 2: 'Clay soil', 3: 'Red soil'}

test_ids_path = Path(r"../data/data/soil_classification-2025") / 'test_ids.csv'

test_id = pd.read_csv(test_ids_path)
test_id.head()

,image_id
0,img_cdf80d6f.jpeg
1,img_c0142a80.jpg
2,img_91168fb0.jpg
3,img_9822190f.jpg
4,img_e5fc436c.jpeg


In [12]:
labels = []
device = "cuda" if torch.cuda.is_available() else 'cpu'

model_vo = TinyVGG(3,10,4).to(device)

state_dict = torch.load('best_model.pth')
model_vo.load_state_dict(state_dict)

for param in model_vo.parameters():
    param.requires_grad = False

In [16]:
   
model_vo.eval()
for i, (_, rows) in enumerate(tqdm(test_id.iterrows(), total=len(test_id), desc="Processing images")):
    img_id = rows['image_id']
    img_path = Path(r"../data/data/soil_classification-2025") / "test" / img_id
    image = Image.open(img_path).convert('RGB')
    t_image = transform(image)

    b_image = t_image.unsqueeze(0).to(device)

    with torch.no_grad():
        y_logits = model_vo(b_image)

    y_pred_class = torch.argmax(y_logits, dim=1).item()

    labels.append(idx_to_class[y_pred_class])


Processing images: 100%|██████████| 341/341 [00:06<00:00, 51.58it/s]


In [17]:
test_id['soil_type'] = labels

In [20]:
test_id.to_csv('submission.csv',index=False)

In [22]:
test_id.head()

,image_id,soil_type
0,img_cdf80d6f.jpeg,Alluvial soil
1,img_c0142a80.jpg,Alluvial soil
2,img_91168fb0.jpg,Alluvial soil
3,img_9822190f.jpg,Alluvial soil
4,img_e5fc436c.jpeg,Clay soil
